### This is the code for using 2 or three consicutive words (as phrase) for prediction. For the main implemintation using only words, please see the other notebook named project5_final

In [1]:
import seaborn as sns
import matplotlib as plt
import pandas as pd
import numpy as np
import nltk
from statistics import mode 

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn import preprocessing

In [2]:
data = pd.read_csv('debate_transcripts.csv')
data.head()

,debate_name,debate_section,speaker,speech,speaking_time_seconds
0,New Hampshire Democratic Debate Transcript,Part 1,George S.,"Candidates, welcome. Vice President Biden, the...",18.0
1,New Hampshire Democratic Debate Transcript,Part 1,Joe Biden,"Oh, they didn’t miss anything. It’s a long rac...",36.0
2,New Hampshire Democratic Debate Transcript,Part 1,George S.,Why are Senator Sanders and Mayor Buttigieg to...,4.0
3,New Hampshire Democratic Debate Transcript,Part 1,Joe Biden,"Well, you know that with regard to Senator San...",41.0
4,New Hampshire Democratic Debate Transcript,Part 1,George S.,"Senator Sanders, let me give you the chance to...",21.0


In [3]:
data.speaker.unique()

array(['George S.', 'Joe Biden', 'Bernie Sanders', 'Amy Klobuchar',
       'Tom Steyer', 'Andrew Yang', 'Elizabeth Warren', 'Pete Buttigieg',
       'Speaker 1', 'Linsey Davis', 'David Muir', 'Monica Hernandez',
       'Adam Sexton', 'Devin Dwyer', 'Rachel Scott', 'Announcer',
       'Wolf Blitzer', 'Abby Phillips', 'B. Pfannenstiel', 'Moderator 1',
       'Moderator 2', 'Brianne P.', 'Judy Woodruff', 'Amy Walter',
       'Stephanie Sy', 'Speaker 2', 'Tim Alberta', 'Amna Nawaz',
       'Yamiche A.', 'Rachel Maddow', 'Andrea Mitchell', 'Kamala Harris',
       'Cory Booker', 'Kristen Welker', 'Ashley Parker', 'Tulsi Gabbard',
       'Speaker 3', 'Anderson Cooper', 'Erin Burnett', 'Marc Lacey',
       'Julian Castro', 'Beto O’Rourke', 'A. Cooper', 'Jake Tapper',
       'Voiceover', 'Jorge Ramos', 'Sec. Castro', 'Speaker 13',
       'Dana Bash', 'Bill de Blasio', 'Michael Bennet', 'Jay Inslee',
       'Kirsten Gillibrand', 'Don Lemon', 'Crowd', 'Kirseten Gillibrand',
       'Moderator', 'S

In [4]:
# Michael bloomberg not included because Nevada debate not included in data set
# Other current candidates have been removed due to not being in the latest debates (like Tulsi Gabbard)
# Candidates that have dropped out are not included

top_candidates = ['Joe Biden', 'Bernie Sanders', 'Amy Klobuchar', 'Tom Steyer', 'Elizabeth Warren', 
                  'Pete Buttigieg']

In [5]:
data = data.drop(columns="debate_section")
data.head()

,debate_name,speaker,speech,speaking_time_seconds
0,New Hampshire Democratic Debate Transcript,George S.,"Candidates, welcome. Vice President Biden, the...",18.0
1,New Hampshire Democratic Debate Transcript,Joe Biden,"Oh, they didn’t miss anything. It’s a long rac...",36.0
2,New Hampshire Democratic Debate Transcript,George S.,Why are Senator Sanders and Mayor Buttigieg to...,4.0
3,New Hampshire Democratic Debate Transcript,Joe Biden,"Well, you know that with regard to Senator San...",41.0
4,New Hampshire Democratic Debate Transcript,George S.,"Senator Sanders, let me give you the chance to...",21.0


In [6]:
data = data.loc[data['speaker'].isin(top_candidates)]
data.head()

,debate_name,speaker,speech,speaking_time_seconds
1,New Hampshire Democratic Debate Transcript,Joe Biden,"Oh, they didn’t miss anything. It’s a long rac...",36.0
3,New Hampshire Democratic Debate Transcript,Joe Biden,"Well, you know that with regard to Senator San...",41.0
5,New Hampshire Democratic Debate Transcript,Bernie Sanders,Because Donald Trump lies all the time. It doe...,41.0
6,New Hampshire Democratic Debate Transcript,Bernie Sanders,I believe that the way we beat Trump is by hav...,39.0
8,New Hampshire Democratic Debate Transcript,Bernie Sanders,That’s true. And that’s the disappointment and...,23.0


In [7]:
##Taking off Test Data (10% in random, training in other 10%)
import math
data = data.sample(frac=1).reset_index(drop=True)
data.head()
numData =len(data)
testData = data[:(math.floor(numData*0.1))]
data = data[(math.floor(numData*0.1)+1):]

## Get most frequent words for each candidate

In [8]:
freqWordsPair = {}
wordCountsPair = {}

for candidate in top_candidates:
    candidate_data = data.loc[data['speaker'] == candidate]
    freqWordsPair[candidate] = {}
    words = 0
    for line in candidate_data['speech']:
        line = line.lower().replace(",", "").replace(".", "").replace("?", "").replace("-", "")
        wordList = line.split()
        line = zip(wordList, wordList[1:])
        for word in line:
            words += 1
#             word = word.lower().replace(",", "").replace(".", "").replace("?", "").replace("-", "")
            if word in (freqWordsPair[candidate]):
                freqWordsPair[candidate][word] += 1
            else:
                freqWordsPair[candidate][word] = 1
    wordCountsPair[candidate] = words
                
    freqWordsPair[candidate] = sorted(freqWordsPair[candidate].items(), key = lambda kv:(kv[1], kv[0]))
    freqWordsPair[candidate].reverse()
                
# for candidate in freqWords:
#     print(candidate, freqWords[candidate][:50])
wordCountsPair

{'Joe Biden': 21184,
 'Bernie Sanders': 16636,
 'Amy Klobuchar': 17036,
 'Tom Steyer': 8547,
 'Elizabeth Warren': 20373,
 'Pete Buttigieg': 18173}

In [9]:
# print(freqWordsPair['Elizabeth Warren'])

In [10]:
for candidate in freqWordsPair:
    print("----- ", candidate, " -----")
#     topWordsVar = candidate.substr(0, text.indexOf(' '))
#     topWordsVar = topWordsVar +'TopWords'
#     print(topWordsVar)
    print(freqWordsPair[candidate][:5])


-----  Joe Biden  -----
[(('of', 'the'), 124), (('to', 'be'), 93), (('in', 'the'), 92), (('going', 'to'), 92), (('have', 'to'), 91)]
-----  Bernie Sanders  -----
[(('of', 'the'), 112), (('we', 'have'), 97), (('this', 'country'), 67), (('in', 'the'), 58), (('on', 'the'), 56)]
-----  Amy Klobuchar  -----
[(('and', 'i'), 101), (('i', 'think'), 89), (('in', 'the'), 74), (('of', 'the'), 70), (('going', 'to'), 61)]
-----  Tom Steyer  -----
[(('going', 'to'), 75), (('have', 'to'), 62), (('to', 'have'), 41), (('of', 'the'), 37), (('we’re', 'going'), 35)]
-----  Elizabeth Warren  -----
[(('we', 'need'), 101), (('going', 'to'), 96), (('need', 'to'), 89), (('we', 'have'), 77), (('of', 'the'), 71)]
-----  Pete Buttigieg  -----
[(('of', 'the'), 107), (('in', 'the'), 76), (('to', 'be'), 67), (('going', 'to'), 62), (('this', 'is'), 51)]


In [11]:
# from functools import reduce
# for candidates in 
# reduce(set.intersection, [set(l_) for l_ in l])

In [12]:
wordProportions = {}

for candidate in top_candidates:
    wordProportions[candidate] = {}
    for word in freqWordsPair[candidate]:
        wordProportions[candidate][word[0]] = word[1]/wordCountsPair[candidate]



In [13]:
# wordProportions['Bernie Sanders']

## Idea: loop through each word in a sentence and add the probability of each word to a candidate's score (using naive bayes) - the candidate with the highest score is guessed as the speaker

In [14]:
data.head()

,debate_name,speaker,speech,speaking_time_seconds
161,Transcript of July Democratic Debate 2nd Round...,Elizabeth Warren,And go up for corporations. For middle class f...,3.0
162,September Houston Democratic Debate Transcript...,Joe Biden,Thank you.,1.0
163,Transcript of July Democratic Debate 2nd Round...,Amy Klobuchar,"Yes. This isn’t just about a system, or it’s n...",43.0
164,Transcript from Night 2 of the First 2019 June...,Joe Biden,A real 30 seconds?,2.0
165,December Democratic Debate Transcript: Sixth D...,Joe Biden,Bibi Netanyahu and I know one another well. He...,18.0


In [15]:
data['speaker'].value_counts() / len(data)

Joe Biden           0.216609
Elizabeth Warren    0.200000
Bernie Sanders      0.189619
Pete Buttigieg      0.172318
Amy Klobuchar       0.152941
Tom Steyer          0.068512
Name: speaker, dtype: float64

In [16]:
speakerProbabilities = {'Joe Biden':0.217933,
                        'Elizabeth Warren':0.200498,
                        'Bernie Sanders':0.186800,
                        'Pete Buttigieg':0.171233,
                        'Amy Klobuchar':0.155044,
                        'Tom Steyer':0.068493}

## It might be easier to just loop through all words and create a new dataframe with each word and candidate label to work with the built-in naive bayes

In [17]:
tmpList = []
tmpList.append([])
tmpList.append([])

for candidate in top_candidates:
    candidate_data = data.loc[data['speaker'] == candidate]
    for line in candidate_data['speech']:
        line = line.lower().replace(",", "").replace(".", "").replace("?", "").replace("-", "")
        wordList = line.split()
        line = zip(wordList, wordList[1:])
        for word in line:
            tmpList[0].append(word)
            tmpList[1].append(candidate)

allWords = pd.DataFrame(np.array(tmpList).T,columns=['word','candidate'])
# len(allWords)

In [18]:
allWords.head()

,word,candidate
0,"(thank, you)",Joe Biden
1,"(a, real)",Joe Biden
2,"(real, 30)",Joe Biden
3,"(30, seconds)",Joe Biden
4,"(bibi, netanyahu)",Joe Biden


In [19]:
len(allWords)

101949

In [20]:
allWords['candidate'].value_counts()

Joe Biden           21184
Elizabeth Warren    20373
Pete Buttigieg      18173
Amy Klobuchar       17036
Bernie Sanders      16636
Tom Steyer           8547
Name: candidate, dtype: int64

In [21]:
#Redundent because Train-test data is already splited from dataframe 
# X_train, X_test, y_train, y_test = train_test_split(allWords['word'], allWords['candidate'], test_size=0.25, random_state=0)

In [22]:
allWords.dtypes

word         object
candidate    object
dtype: object

In [23]:
# see example code
import random
# allWords = allWords.sample(frac=1) # words have been shuffled
# allWords.head()

tmp = []

for i in range(len(allWords)):
    tmp.append(({'word':allWords['word'][i]},allWords['candidate'][i]))

random.shuffle(tmp)
# print(type(tmp))

training = tmp[:int(len(allWords) * 0.9)]
test = tmp[int(len(allWords) * 0.1):]

model_0 = nltk.NaiveBayesClassifier.train(training) ### DIFFERENCE ######

In [24]:
print(model_0.classify({'word': ('is', 'part')}))
print(model_0.classify({'word':('to', 'hear')}))
print(model_0.classify({'word':('know', 'we')}))
print(model_0.classify({'word':('we', 'saw')}))
print(model_0.classify({'word':('part', 'of')}))
# print(model_0.classify({'word:':''}))

Pete Buttigieg
Amy Klobuchar
Elizabeth Warren
Pete Buttigieg
Pete Buttigieg


In [25]:
# from example code
print(nltk.classify.accuracy(model_0, test))

0.6257097705847093


## Given 1 word, our accuracy is almost 60-70%

## Now, let's see if a vote-type system works better (count up predictions for a speech and see which candidate was most-often predicted)

In [26]:
# test

In [27]:
speeches = []


for candidate in top_candidates:
    candidate_data = testData.loc[testData['speaker'] == candidate]
    for line in candidate_data['speech']:
        line = line.lower().replace(",", "").replace(".", "").replace("?", "").replace("-", "").replace("\"", "")
        wordList = line.split()
        line = zip(wordList, wordList[1:], wordList[2:])
        words = []
        for word in line:
            words.append(({'word':word}, candidate))
        speeches.append(words)
        
# tmp[0][0][1]

In [28]:
#To-Do: Print Collisions

from collections import Counter
# collision = 0
def get_first_mode(a):
    c = Counter(a)  
    mode_count = max(c.values())
    mode = {key for key, count in c.items() if count == mode_count}
    first_mode = next(x for x in a if x in mode)
    return first_mode

In [29]:

for sp in speeches:
    if len(sp)<1:
        speeches.remove(sp)

In [30]:
for sp in speeches:
    if len(sp)<1:
        speeches.remove(sp)
print(len(speeches))

155


In [31]:
##To-DO: Print number of time with zero hits and defulting to Joe
num_def = 0
num_correct = 0.0
random.shuffle(speeches)
for speech in speeches:

    word_predictions = []
    for word in speech:
#         print(word)
        word_predictions.append(model_0.classify(word[0]))
    
    if(len(word_predictions)):
        speech_prediction = mode(word_predictions)
 
    if speech_prediction == speech[0][1]:
        num_correct +=1

print(num_correct / len(speeches))


0.22580645161290322


## Cool, we got it up to ~23%, which I'll consider pretty good considering there are six candidates using a lot of similar language and talking about similar topics

Maybe accuracy could be improved by removing common words? - there seems to be a bias in these words to Joe Biden since he had the most speaking time

In [32]:
#Redundent in word pair

# common_words = ["medicare","a","about","above","after","again","against","ain","all","am","an","and","any","are","aren","aren't","as","at","be","because","been","before","being","below","between","both","but","by","can","couldn","couldn't","d","did","didn","didn't","do","does","doesn","doesn't","doing","don","don't","down","during","each","few","for","from","further","had","hadn","hadn't","has","hasn","hasn't","have","haven","haven't","having","he","her","here","hers","herself","him","himself","his","how","i","if","in","into","is","isn","isn't","it","it's","its","itself","just","ll","m","ma","me","mightn","mightn't","more","most","mustn","mustn't","my","myself","needn","needn't","no","nor","not","now","o","of","off","on","once","only","or","other","our","ours","ourselves","out","over","own","re","s","same","shan","shan't","she","she's","should","should've","shouldn","shouldn't","so","some","such","t","than","that","that'll","the","their","theirs","them","themselves","then","there","these","they","this","those","through","to","too","under","until","up","ve","very","was","wasn","wasn't","we","were","weren","weren't","what","when","where","which","while","who","whom","why","will","with","won","won't","wouldn","wouldn't","y","you","you'd","you'll","you're","you've","your","yours","yourself","yourselves","could","he'd","he'll","he's","here's","how's","i'd","i'll","i'm","i've","let's","ought","she'd","she'll","that's","there's","they'd","they'll","they're","they've","we'd","we'll","we're","we've","what's","when's","where's","who's","why's","would","able","abst","accordance","according","accordingly","across","act","actually","added","adj","affected","affecting","affects","afterwards","ah","almost","alone","along","already","also","although","always","among","amongst","announce","another","anybody","anyhow","anymore","anyone","anything","anyway","anyways","anywhere","apparently","approximately","arent","arise","around","aside","ask","asking","auth","available","away","awfully","b","back","became","become","becomes","becoming","beforehand","begin","beginning","beginnings","begins","behind","believe","beside","besides","beyond","biol","brief","briefly","c","ca","came","cannot","can't","cause","causes","certain","certainly","co","com","come","comes","contain","containing","contains","couldnt","date","different","done","downwards","due","e","ed","edu","effect","eg","eight","eighty","either","else","elsewhere","end","ending","enough","especially","et","etc","even","ever","every","everybody","everyone","everything","everywhere","ex","except","f","far","ff","fifth","first","five","fix","followed","following","follows","former","formerly","forth","found","four","furthermore","g","gave","get","gets","getting","give","given","gives","giving","go","goes","gone","got","gotten","h","happens","hardly","hed","hence","hereafter","hereby","herein","heres","hereupon","hes","hi","hid","hither","home","howbeit","however","hundred","id","ie","im","immediate","immediately","importance","important","inc","indeed","index","information","instead","invention","inward","itd","it'll","j","k","keep","keeps","kept","kg","km","know","known","knows","l","largely","last","lately","later","latter","latterly","least","less","lest","let","lets","like","liked","likely","line","little","'ll","look","looking","looks","ltd","made","mainly","make","makes","many","may","maybe","mean","means","meantime","meanwhile","merely","mg","might","million","miss","ml","moreover","mostly","mr","mrs","much","mug","must","n","na","name","namely","nay","nd","near","nearly","necessarily","necessary","need","needs","neither","never","nevertheless","new","next","nine","ninety","nobody","non","none","nonetheless","noone","normally","nos","noted","nothing","nowhere","obtain","obtained","obviously","often","oh","ok","okay","old","omitted","one","ones","onto","ord","others","otherwise","outside","overall","owing","p","page","pages","part","particular","particularly","past","per","perhaps","placed","please","plus","poorly","possible","possibly","potentially","pp","predominantly","present","previously","primarily","probably","promptly","proud","provides","put","q","que","quickly","quite","qv","r","ran","rather","rd","readily","really","recent","recently","ref","refs","regarding","regardless","regards","related","relatively","research","respectively","resulted","resulting","results","right","run","said","saw","say","saying","says","sec","section","see","seeing","seem","seemed","seeming","seems","seen","self","selves","sent","seven","several","shall","shed","shes","show","showed","shown","showns","shows","significant","significantly","similar","similarly","since","six","slightly","somebody","somehow","someone","somethan","something","sometime","sometimes","somewhat","somewhere","soon","sorry","specifically","specified","specify","specifying","still","stop","strongly","sub","substantially","successfully","sufficiently","suggest","sup","sure","take","taken","taking","tell","tends","th","thank","thanks","thanx","thats","that've","thence","thereafter","thereby","thered","therefore","therein","there'll","thereof","therere","theres","thereto","thereupon","there've","theyd","theyre","think","thou","though","thoughh","thousand","throug","throughout","thru","thus","til","tip","together","took","toward","towards","tried","tries","truly","try","trying","ts","twice","two","u","un","unfortunately","unless","unlike","unlikely","unto","upon","ups","us","use","used","useful","usefully","usefulness","uses","using","usually","v","value","various","'ve","via","viz","vol","vols","vs","w","want","wants","wasnt","way","wed","welcome","went","werent","whatever","what'll","whats","whence","whenever","whereafter","whereas","whereby","wherein","wheres","whereupon","wherever","whether","whim","whither","whod","whoever","whole","who'll","whomever","whos","whose","widely","willing","wish","within","without","wont","words","world","wouldnt","www","x","yes","yet","youd","youre","z","zero","a's","ain't","allow","allows","apart","appear","appreciate","appropriate","associated","best","better","c'mon","c's","cant","changes","clearly","concerning","consequently","consider","considering","corresponding","course","currently","definitely","described","despite","entirely","exactly","example","going","greetings","hello","help","hopefully","ignored","inasmuch","indicate","indicated","indicates","inner","insofar","it'd","keep","keeps","novel","presumably","reasonably","second","secondly","sensible","serious","seriously","sure","t's","third","thorough","thoroughly","three","well","wonder"]
# num_correct = 0
# for speech in speeches:
#     word_predictions = []
#     for word in speech:
#         if word[0]['word'] not in common_words:
#             word_predictions.append(model_0.classify(word[0]))

#     if (len(word_predictions)):
#         speech_prediction = mode(word_predictions)
#         if speech_prediction == speech[0][1]:
#             num_correct +=1
        
# print(num_correct / len(speeches))

## TODO: predict nevada debate speeches (and possibly south carolina)

We could scrape from https://www.rev.com/blog/transcripts/democratic-debate-transcript-las-vegas-nevada-debate
and https://www.rev.com/blog/transcripts/south-carolina-democratic-debate-transcript-february-democratic-debate

## Now lets look as three word pair for the size of phrase

In [33]:
data = pd.read_csv('debate_transcripts.csv')
data = data.drop(columns="debate_section")
data.head()

,debate_name,speaker,speech,speaking_time_seconds
0,New Hampshire Democratic Debate Transcript,George S.,"Candidates, welcome. Vice President Biden, the...",18.0
1,New Hampshire Democratic Debate Transcript,Joe Biden,"Oh, they didn’t miss anything. It’s a long rac...",36.0
2,New Hampshire Democratic Debate Transcript,George S.,Why are Senator Sanders and Mayor Buttigieg to...,4.0
3,New Hampshire Democratic Debate Transcript,Joe Biden,"Well, you know that with regard to Senator San...",41.0
4,New Hampshire Democratic Debate Transcript,George S.,"Senator Sanders, let me give you the chance to...",21.0


In [34]:
data = data.loc[data['speaker'].isin(top_candidates)]
data.head()
##Taking off Test Data (10% in random, training in other 90%)
import math
data = data.sample(frac=1).reset_index(drop=True)
data.head()
numData =len(data)
testData = data[:(math.floor(numData*0.1))]
data = data[(math.floor(numData*0.1)+1):]

In [35]:
freqWordsPair = {}
wordCountsPair = {}

for candidate in top_candidates:
    candidate_data = data.loc[data['speaker'] == candidate]
    freqWordsPair[candidate] = {}
    words = 0
    for line in candidate_data['speech']:
        line = line.lower().replace(",", "").replace(".", "").replace("?", "").replace("-", "")
        wordList = line.split()
        line = zip(wordList, wordList[1:], wordList[2:])
        for word in line:
            words += 1
#             word = word.lower().replace(",", "").replace(".", "").replace("?", "").replace("-", "")
            if word in (freqWordsPair[candidate]):
                freqWordsPair[candidate][word] += 1
            else:
                freqWordsPair[candidate][word] = 1
    wordCountsPair[candidate] = words
                
    freqWordsPair[candidate] = sorted(freqWordsPair[candidate].items(), key = lambda kv:(kv[1], kv[0]))
    freqWordsPair[candidate].reverse()
                
# for candidate in freqWords:
#     print(candidate, freqWords[candidate][:5])
# wordCountsPair

In [36]:
for candidate in freqWordsPair:
    print("----- ", candidate, " -----")
#     topWordsVar = candidate.substr(0, text.indexOf(' '))
#     topWordsVar = topWordsVar +'TopWords'
#     print(topWordsVar)
    print(freqWordsPair[candidate][:5])

-----  Joe Biden  -----
[(('we', 'have', 'to'), 57), (('the', 'united', 'states'), 47), (('be', 'able', 'to'), 34), (('to', 'be', 'able'), 25), (('make', 'sure', 'that'), 24)]
-----  Bernie Sanders  -----
[(('medicare', 'for', 'all'), 33), (('the', 'american', 'people'), 32), (('in', 'this', 'country'), 32), (('have', 'got', 'to'), 30), (('we', 'have', 'got'), 29)]
-----  Amy Klobuchar  -----
[(('i', 'want', 'to'), 24), (('we', 'have', 'to'), 23), (('and', 'i', 'think'), 23), (('we', 'need', 'to'), 22), (('when', 'it', 'comes'), 19)]
-----  Tom Steyer  -----
[(('we’re', 'going', 'to'), 34), (('going', 'to', 'have'), 30), (('to', 'have', 'to'), 28), (('on', 'this', 'stage'), 27), (('we', 'have', 'to'), 23)]
-----  Elizabeth Warren  -----
[(('we', 'need', 'to'), 76), (('we', 'have', 'to'), 33), (('in', 'this', 'country'), 29), (('the', 'united', 'states'), 27), (('i', 'want', 'to'), 27)]
-----  Pete Buttigieg  -----
[(('the', 'american', 'people'), 22), (('we', 'need', 'to'), 20), (('we’

In [37]:
wordProportions = {}

for candidate in top_candidates:
    wordProportions[candidate] = {}
    for word in freqWordsPair[candidate]:
        wordProportions[candidate][word[0]] = word[1]/wordCountsPair[candidate]



In [38]:
data['speaker'].value_counts() / len(data)

Joe Biden           0.212457
Elizabeth Warren    0.197232
Bernie Sanders      0.191696
Pete Buttigieg      0.173010
Amy Klobuchar       0.155709
Tom Steyer          0.069896
Name: speaker, dtype: float64

In [39]:
tmpList = []
tmpList.append([])
tmpList.append([])

for candidate in top_candidates:
    candidate_data = data.loc[data['speaker'] == candidate]
    for line in candidate_data['speech']:
        line = line.lower().replace(",", "").replace(".", "").replace("?", "").replace("-", "")
        wordList = line.split()
        line = zip(wordList, wordList[1:], wordList[2:])
        for word in line:
            tmpList[0].append(word)
            tmpList[1].append(candidate)

allWords = pd.DataFrame(np.array(tmpList).T,columns=['word','candidate'])
# len(allWords)

In [40]:
# see example code
import random

tmp = []

for i in range(len(allWords)):
    tmp.append(({'word':allWords['word'][i]},allWords['candidate'][i]))

random.shuffle(tmp)

training = tmp[:int(len(allWords) * 0.9)]
test = tmp[int(len(allWords) * 0.1):]

model_0 = nltk.NaiveBayesClassifier.train(training) ### DIFFERENCE ######

In [41]:
print(model_0.classify({'word': ('is', 'part', 'of')}))
print(model_0.classify({'word':('able', 'to', 'hear')}))
print(model_0.classify({'word':('i', 'know', 'we')}))
print(model_0.classify({'word':('that', 'we', 'saw')}))
print(model_0.classify({'word':('is', 'part', 'of')}))
# print(model_0.classify({'word:':''}))

Pete Buttigieg
Bernie Sanders
Amy Klobuchar
Pete Buttigieg
Pete Buttigieg


In [42]:
# from example code
print(nltk.classify.accuracy(model_0, test))

0.824970866955432


In [43]:
speeches = []

for candidate in top_candidates:
    candidate_data = testData.loc[testData['speaker'] == candidate]
    for line in candidate_data['speech']:
        line = line.lower().replace(",", "").replace(".", "").replace("?", "").replace("-", "").replace("\"", "")
        wordList = line.split()
        line = zip(wordList, wordList[1:], wordList[2:])
        words = []
        for word in line:
            words.append(({'word':word}, candidate))
        speeches.append(words)
        

In [44]:
#Removing empty or speech with less than 3 words
for sp in speeches:
    if len(sp)<1:
        speeches.remove(sp)
for sp in speeches:
    if len(sp)<1:
        speeches.remove(sp)
print(len(speeches))

148


In [45]:
#For event of more than one mode

from collections import Counter
# collision = 0
def get_first_mode(a):
    c = Counter(a)  
    mode_count = max(c.values())
    mode = {key for key, count in c.items() if count == mode_count}
    first_mode = next(x for x in a if x in mode)
    return first_mode

In [46]:
num_def = 0
num_correct = 0.0
random.shuffle(speeches)
for speech in speeches:

    word_predictions = []
    for word in speech:
#         print(word)
        word_predictions.append(model_0.classify(word[0]))
    
    if(len(word_predictions)):
        speech_prediction = get_first_mode(word_predictions)
 
    if speech_prediction == speech[0][1]:
        num_correct +=1

print(num_correct / len(speeches))

0.3108108108108108
